# AlfWorld DQN Training - Google Colab 🤖

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/)

使用 **Double DQN + DistilBERT** 训练 AlfWorld agent

## 📋 实验概述

- **模型**: DistilBERT-base-cased (Embedding层)
- **算法**: Double DQN with Prioritized Replay
- **环境**: AlfWorld TextWorld 环境
- **架构**: 完全按照 alfworld_meta_dqn-master 实现

## 🎯 特点

- ✅ 轻量级（DistilBERT ~300MB）
- ✅ 完整的 DQN 架构（Attention + Recurrent）
- ✅ Prioritized Experience Replay
- ✅ 免费运行（Colab T4 GPU）

## ⏱️ 预计时间

- 环境搭建: ~15 分钟
- 训练 100 episodes: ~2-3 小时
- 完整训练 (1000+ episodes): 1-2 天

## 1. 检查运行环境 🔍

In [ ]:
import sys
import os

# 检查 Colab
IN_COLAB = 'google.colab' in sys.modules
print(f"{'✓' if IN_COLAB else '✗'} Google Colab: {IN_COLAB}")
print(f"Python: {sys.version.split()[0]}\n")

# 检查 GPU
import torch
if torch.cuda.is_available():
    print(f"✓ GPU: {torch.cuda.get_device_name(0)}")
    print(f"  显存: {torch.cuda.get_device_properties(0).total_memory / 1024**3:.1f} GB")
    print(f"  CUDA: {torch.version.cuda}")
else:
    print("❌ 未检测到 GPU！请启用 GPU 运行时")
    print("   运行时 → 更改运行时类型 → GPU")

!free -h | head -2

## 2. 挂载 Google Drive 📁

In [ ]:
if IN_COLAB:
    from google.colab import drive
    drive.mount('/content/drive')

# 项目目录结构
PROJECT_DIR = '/content/drive/MyDrive/AlfWorld_DQN' if IN_COLAB else './alfworld_dqn_workspace'
DATA_DIR = os.path.join(PROJECT_DIR, 'alfworld_data')
RESULTS_DIR = os.path.join(PROJECT_DIR, 'results')
CHECKPOINTS_DIR = os.path.join(PROJECT_DIR, 'checkpoints')

for dir_path in [PROJECT_DIR, DATA_DIR, RESULTS_DIR, CHECKPOINTS_DIR]:
    os.makedirs(dir_path, exist_ok=True)

print("目录结构:")
print(f"  项目: {PROJECT_DIR}")
print(f"  数据: {DATA_DIR}")
print(f"  结果: {RESULTS_DIR}")
print(f"  检查点: {CHECKPOINTS_DIR}")

## 3. 安装依赖包 📦

In [ ]:
print("安装依赖包...\n")

# 1. AlfWorld (完整版)
print("[1/4] 安装 AlfWorld...")
!pip install -q alfworld[full]

# 2. Transformers + PyTorch
print("[2/4] 安装 Transformers...")
!pip install -q transformers>=4.35.0 accelerate sentencepiece

# 3. 其他依赖
print("[3/4] 安装其他工具...")
!pip install -q pandas matplotlib seaborn tqdm pyyaml

# 4. 验证安装
print("[4/4] 验证安装...")
import alfworld
from transformers import DistilBertModel, DistilBertTokenizer
print("  ✓ alfworld", alfworld.__version__)
print("  ✓ transformers")
print("  ✓ torch", torch.__version__)

print("\n✓ 所有依赖已安装")

## 4. 下载 AlfWorld 数据 🗂️

In [ ]:
# 设置环境变量
os.environ['ALFWORLD_DATA'] = DATA_DIR
print(f"ALFWORLD_DATA = {DATA_DIR}\n")

# 检查是否已下载
data_check_path = os.path.join(DATA_DIR, 'json_2.1.1')

if os.path.exists(data_check_path) and os.path.exists(os.path.join(DATA_DIR, 'logic')):
    print("✓ AlfWorld 数据已存在")
    !ls -lh {DATA_DIR}
else:
    print("正在下载 AlfWorld 数据...")
    print("预计大小: ~2GB")
    print("预计时间: 5-10 分钟\n")
    
    !alfworld-download --path {DATA_DIR}
    
    print("\n✓ AlfWorld 数据下载完成")
    !ls -lh {DATA_DIR}

# 验证数据完整性
required_paths = [
    os.path.join(DATA_DIR, 'json_2.1.1/train'),
    os.path.join(DATA_DIR, 'json_2.1.1/valid_seen'),
    os.path.join(DATA_DIR, 'json_2.1.1/valid_unseen'),
    os.path.join(DATA_DIR, 'logic/alfred.pddl'),
    os.path.join(DATA_DIR, 'logic/alfred.twl2')
]

print("\n验证数据完整性:")
all_ok = True
for path in required_paths:
    exists = os.path.exists(path)
    print(f"  {'✓' if exists else '✗'} {os.path.basename(path)}")
    all_ok = all_ok and exists

if all_ok:
    print("\n✓ 所有数据文件完整")
else:
    print("\n⚠️ 某些数据文件缺失")

## 5. 创建配置文件 ⚙️

In [ ]:
import yaml

# 创建 base_config.yaml (简化版)
config = {
    'dataset': {
        'data_path': f"{DATA_DIR}/json_2.1.1/train",
        'eval_id_data_path': f"{DATA_DIR}/json_2.1.1/valid_seen",
        'eval_ood_data_path': f"{DATA_DIR}/json_2.1.1/valid_unseen",
    },
    'logic': {
        'domain': f"{DATA_DIR}/logic/alfred.pddl",
        'grammar': f"{DATA_DIR}/logic/alfred.twl2",
    },
    'env': {
        'type': 'AlfredTWEnv',
        'regen_game_files': False,
        'domain_randomization': False,
        'task_types': [1, 2, 3, 4, 5, 6],
    }
}

config_path = os.path.join(PROJECT_DIR, 'base_config.yaml')
with open(config_path, 'w') as f:
    yaml.dump(config, f, default_flow_style=False)

print(f"✓ 配置文件已创建: {config_path}")
print("\n内容:")
!cat {config_path}

## 6. 加载 DistilBERT 模型 🧠

In [ ]:
from transformers import DistilBertModel, DistilBertTokenizer
import torch

print("加载 DistilBERT (与 meta-dqn 一致)...\n")

# 1. Tokenizer
print("[1/2] 加载 tokenizer...")
tokenizer = DistilBertTokenizer.from_pretrained('distilbert-base-cased')
word2id = tokenizer.get_vocab()
print(f"✓ Tokenizer 完成")
print(f"  词表大小: {len(word2id)}")

# 2. BERT Model (只保留 embeddings 层)
print("\n[2/2] 加载 DistilBERT...")
bert_model = DistilBertModel.from_pretrained('distilbert-base-cased')

# 移除不需要的层（与 meta-dqn 一致）
bert_model.transformer = None
bert_model.encoder = None

# 冻结参数
for param in bert_model.parameters():
    param.requires_grad = False

# 移动到 GPU
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
bert_model = bert_model.to(device)
bert_model.eval()

print(f"✓ DistilBERT 加载完成")
print(f"  参数量: {sum(p.numel() for p in bert_model.parameters()) / 1e6:.1f}M")
print(f"  设备: {device}")

if torch.cuda.is_available():
    print(f"  GPU 显存: {torch.cuda.memory_allocated() / 1024**3:.2f} GB")

## 7. 测试 AlfWorld 环境 🧪

In [ ]:
print("测试 AlfWorld 环境...\n")

import alfworld
import alfworld.agents.environment
import yaml

try:
    # 加载配置
    with open(config_path, 'r') as f:
        config = yaml.safe_load(f)
    
    # 创建环境
    env_type = config['env']['type']
    print(f"环境类型: {env_type}")
    
    env_class = alfworld.agents.environment.get_environment(env_type)
    env = env_class(config, train_eval='eval_out_of_distribution')
    env = env.init_env(batch_size=1)
    
    # 重置环境
    obs, info = env.reset()
    print("✓ 环境初始化成功\n")
    
    print(f"初始观察 (前200字符):")
    print(obs[0][:200])
    print("...\n")
    
    # 检查 admissible commands
    if 'admissible_commands' in info:
        admissible = info['admissible_commands'][0]
        print(f"✓ 候选动作数: {len(admissible)}")
        print(f"  示例动作:")
        for i, action in enumerate(admissible[:5]):
            print(f"    {i+1}. {action}")
    
    env.close()
    print("\n✓ AlfWorld 环境测试通过")
    
except Exception as e:
    print(f"❌ 错误: {e}")
    import traceback
    traceback.print_exc()

## 8. DQN 网络架构实现 🏗️

包含完整的 Layer 组件和 Policy 网络

In [ ]:
# 将 alfworld_dqn_train.py 的代码嵌入到 notebook
# 这样可以直接在 Colab 中运行，无需外部文件

import torch
import torch.nn as nn
import torch.nn.functional as F
import numpy as np

# ======================== Layer Components ========================

def compute_mask(x):
    """计算 padding mask"""
    mask = torch.ne(x, 0).float()
    if x.is_cuda:
        mask = mask.cuda()
    return mask


def masked_mean(x, m=None, dim=1):
    """带 mask 的平均池化"""
    if m is None:
        return torch.mean(x, dim=dim)
    x = x * m.unsqueeze(-1)
    mask_sum = torch.sum(m, dim=-1)
    tmp = torch.eq(mask_sum, 0).float()
    if x.is_cuda:
        tmp = tmp.cuda()
    mask_sum = mask_sum + tmp
    res = torch.sum(x, dim=dim)
    res = res / mask_sum.unsqueeze(-1)
    return res


def masked_softmax(x, m=None, axis=-1):
    """带 mask 的 softmax"""
    x = torch.clamp(x, min=-15.0, max=15.0)
    if m is not None:
        m = m.float()
        x = x * m
    e_x = torch.exp(x - torch.max(x, dim=axis, keepdim=True)[0])
    if m is not None:
        e_x = e_x * m
    softmax = e_x / (torch.sum(e_x, dim=axis, keepdim=True) + 1e-6)
    return softmax


class SelfAttention(nn.Module):
    """Multi-head Self-Attention"""
    def __init__(self, block_hidden_dim, n_head, dropout):
        super().__init__()
        self.block_hidden_dim = block_hidden_dim
        self.n_head = n_head
        self.dropout = dropout
        assert block_hidden_dim % n_head == 0
        self.d_k = block_hidden_dim // n_head

        self.linears = nn.ModuleList([nn.Linear(block_hidden_dim, block_hidden_dim) for _ in range(4)])

    def forward(self, query, mask, key, value):
        batch_size = query.size(0)

        query, key, value = [l(x).view(batch_size, -1, self.n_head, self.d_k).transpose(1, 2)
                            for l, x in zip(self.linears, (query, key, value))]

        scores = torch.matmul(query, key.transpose(-2, -1)) / np.sqrt(self.d_k)

        if mask is not None:
            mask = mask.unsqueeze(1)
            scores = scores * mask + -1e9 * (1 - mask)

        attn = F.softmax(scores, dim=-1)
        attn = F.dropout(attn, p=self.dropout, training=self.training)

        out = torch.matmul(attn, value)
        out = out.transpose(1, 2).contiguous().view(batch_size, -1, self.block_hidden_dim)
        out = self.linears[-1](out)

        return out, attn


class CQAttention(nn.Module):
    """Context-Query Attention"""
    def __init__(self, block_hidden_dim, dropout=0.1):
        super().__init__()
        self.dropout = dropout
        w = torch.empty(block_hidden_dim * 3)
        nn.init.xavier_uniform_(w)
        self.w = nn.Parameter(w)

    def forward(self, C, Q, c_mask, q_mask):
        batch_size, c_len, q_len = C.size(0), C.size(1), Q.size(1)

        C_expanded = C.unsqueeze(2).expand(-1, -1, q_len, -1)
        Q_expanded = Q.unsqueeze(1).expand(-1, c_len, -1, -1)
        CQ = C_expanded * Q_expanded

        S_inputs = torch.cat([C_expanded, Q_expanded, CQ], dim=-1)
        S = torch.matmul(S_inputs, self.w)

        S1 = masked_softmax(S, m=q_mask.unsqueeze(1), axis=2)
        A = torch.bmm(S1, Q)

        S2 = masked_softmax(S, m=c_mask.unsqueeze(2), axis=1)
        S2_max, _ = torch.max(S2, dim=1, keepdim=True)
        B = torch.bmm(S2_max, Q).expand(-1, c_len, -1)

        out = torch.cat([C, A, C * A, C * B], dim=-1)

        return out


def PosEncoder(x):
    """Position Encoding"""
    batch_size, seq_len, d_model = x.size()
    pos = torch.arange(0, seq_len).unsqueeze(0).expand(batch_size, -1).float()
    if x.is_cuda:
        pos = pos.cuda()

    div_term = torch.exp(torch.arange(0, d_model, 2).float() * -(np.log(10000.0) / d_model))
    if x.is_cuda:
        div_term = div_term.cuda()

    pe = torch.zeros(batch_size, seq_len, d_model)
    if x.is_cuda:
        pe = pe.cuda()

    pe[:, :, 0::2] = torch.sin(pos.unsqueeze(-1) * div_term)
    pe[:, :, 1::2] = torch.cos(pos.unsqueeze(-1) * div_term)

    return x + pe


class DepthwiseSeparableConv(nn.Module):
    """Depthwise Separable Convolution"""
    def __init__(self, in_ch, out_ch, k, bias=True):
        super().__init__()
        self.depthwise_conv = nn.Conv1d(in_channels=in_ch, out_channels=in_ch, kernel_size=k, groups=in_ch, padding=k // 2, bias=False)
        self.pointwise_conv = nn.Conv1d(in_channels=in_ch, out_channels=out_ch, kernel_size=1, padding=0, bias=bias)

    def forward(self, x, mask):
        x = x.transpose(1, 2)
        x = self.depthwise_conv(x)
        x = self.pointwise_conv(x)
        x = x.transpose(1, 2)
        x = x * mask.unsqueeze(-1)
        return x


class EncoderBlock(nn.Module):
    """Encoder Block"""
    def __init__(self, conv_num, ch_num, k, block_hidden_dim, n_head, dropout):
        super().__init__()
        self.dropout = dropout
        self.convs = nn.ModuleList([DepthwiseSeparableConv(ch_num, ch_num, k) for _ in range(conv_num)])
        self.self_att = SelfAttention(block_hidden_dim, n_head, dropout)
        self.FFN_1 = nn.Linear(ch_num, ch_num)
        self.FFN_2 = nn.Linear(ch_num, ch_num)
        self.norm_C = nn.ModuleList([nn.LayerNorm(block_hidden_dim) for _ in range(conv_num)])
        self.norm_1 = nn.LayerNorm(block_hidden_dim)
        self.norm_2 = nn.LayerNorm(block_hidden_dim)
        self.conv_num = conv_num

    def forward(self, x, mask, squared_mask, l, blks):
        total_layers = (self.conv_num + 2) * blks
        out = PosEncoder(x)
        out = out * mask.unsqueeze(-1)

        for i, conv in enumerate(self.convs):
            res = out
            out = self.norm_C[i](out)
            if (i) % 2 == 0:
                out = F.dropout(out, p=self.dropout, training=self.training)
            out = conv(out, mask)
            out = self.layer_dropout(out, res, self.dropout * float(l) / total_layers)
            out = out * mask.unsqueeze(-1)
            l += 1

        res = out
        out = self.norm_1(out)
        out = F.dropout(out, p=self.dropout, training=self.training)
        out, _ = self.self_att(out, squared_mask, out, out)
        out = self.layer_dropout(out, res, self.dropout * float(l) / total_layers)
        out = out * mask.unsqueeze(-1)
        l += 1

        res = out
        out = self.norm_2(out)
        out = F.dropout(out, p=self.dropout, training=self.training)
        out = self.FFN_1(out)
        out = torch.relu(out)
        out = self.FFN_2(out)
        out = self.layer_dropout(out, res, self.dropout * float(l) / total_layers)
        out = out * mask.unsqueeze(-1)
        l += 1

        return out

    def layer_dropout(self, inputs, residual, dropout):
        if self.training == True:
            pred = torch.empty(1).uniform_(0, 1) < dropout
            if pred:
                return residual
            else:
                return F.dropout(inputs, dropout, training=self.training) + residual
        else:
            return inputs + residual


print("✓ Layer 组件已定义")

In [ ]:
# ======================== Policy Network ========================

class Policy(nn.Module):
    """DQN Policy Network"""
    def __init__(self, bert_model, word_vocab_size, config):
        super().__init__()
        self.bert_model = bert_model
        self.word_vocab_size = word_vocab_size
        self.config = config

        self.encoder_layers = config['encoder_layers']
        self.encoder_conv_num = config['encoder_conv_num']
        self.block_hidden_dim = config['block_hidden_dim']
        self.n_heads = config['n_heads']
        self.block_dropout = config['block_dropout']
        self.dropout = config['dropout']
        self.recurrent = config['recurrent']

        self._def_layers()

    def _def_layers(self):
        BERT_EMBEDDING_SIZE = 768

        self.word_embedding_prj = nn.Linear(BERT_EMBEDDING_SIZE, self.block_hidden_dim, bias=False)

        self.encoder = nn.ModuleList([
            EncoderBlock(
                conv_num=self.encoder_conv_num,
                ch_num=self.block_hidden_dim,
                k=5,
                block_hidden_dim=self.block_hidden_dim,
                n_head=self.n_heads,
                dropout=self.block_dropout
            ) for _ in range(self.encoder_layers)
        ])

        self.aggregation_attention = CQAttention(block_hidden_dim=self.block_hidden_dim, dropout=self.dropout)
        self.aggregation_attention_proj = nn.Linear(self.block_hidden_dim * 4, self.block_hidden_dim)

        if self.recurrent:
            self.rnncell = nn.GRUCell(self.block_hidden_dim, self.block_hidden_dim)
            self.dynamics_aggregation = nn.Linear(self.block_hidden_dim * 2, self.block_hidden_dim)
        else:
            self.rnncell, self.dynamics_aggregation = None, None

        self.action_scorer_linear_1 = nn.Linear(self.block_hidden_dim * 2, self.block_hidden_dim)
        self.action_scorer_linear_2 = nn.Linear(self.block_hidden_dim, 1)
        self.action_scorer_extra_self_attention = SelfAttention(self.block_hidden_dim, self.n_heads, self.dropout)
        self.action_scorer_extra_linear = nn.Linear(self.block_hidden_dim, self.block_hidden_dim)

    def get_bert_embeddings(self, _input_words, _input_masks):
        if _input_words.size(1) > 512:
            seg_length = 500
            outputs = []
            num_batch = (_input_words.size(1) + seg_length - 1) // seg_length
            for i in range(num_batch):
                batch_input = _input_words[:, i * seg_length: (i + 1) * seg_length]
                batch_mask = _input_masks[:, i * seg_length: (i + 1) * seg_length]
                out = self.get_bert_embeddings(batch_input, batch_mask)
                outputs.append(out)
            return torch.cat(outputs, 1)

        with torch.no_grad():
            res = self.bert_model.embeddings(_input_words)
            res = res * _input_masks.unsqueeze(-1)
        return res

    def embed(self, input_words, input_word_masks):
        word_embeddings = self.get_bert_embeddings(input_words, input_word_masks)
        word_embeddings = word_embeddings * input_word_masks.unsqueeze(-1)
        word_embeddings = self.word_embedding_prj(word_embeddings)
        word_embeddings = word_embeddings * input_word_masks.unsqueeze(-1)
        return word_embeddings

    def encode_text(self, input_word_ids):
        input_word_masks = compute_mask(input_word_ids)
        embeddings = self.embed(input_word_ids, input_word_masks)

        squared_mask = torch.bmm(input_word_masks.unsqueeze(-1), input_word_masks.unsqueeze(1))
        encoding_sequence = embeddings

        for i in range(self.encoder_layers):
            encoding_sequence = self.encoder[i](
                encoding_sequence, input_word_masks, squared_mask,
                i * (self.encoder_conv_num + 2) + 1, self.encoder_layers
            )

        return encoding_sequence, input_word_masks

    def aggretate_information(self, h_obs, obs_mask, h_td, td_mask):
        aggregated_obs_representation = self.aggregation_attention(h_obs, h_td, obs_mask, td_mask)
        aggregated_obs_representation = self.aggregation_attention_proj(aggregated_obs_representation)
        aggregated_obs_representation = torch.tanh(aggregated_obs_representation)
        aggregated_obs_representation = aggregated_obs_representation * obs_mask.unsqueeze(-1)
        return aggregated_obs_representation

    def masked_mean(self, h_obs, obs_mask):
        _mask = torch.sum(obs_mask, -1)
        obs_representations = torch.sum(h_obs, -2)
        tmp = torch.eq(_mask, 0).float()
        if obs_representations.is_cuda:
            tmp = tmp.cuda()
        _mask = _mask + tmp
        obs_representations = obs_representations / _mask.unsqueeze(-1)
        return obs_representations

    def score_actions(self, candidate_representations, cand_mask, h_obs, obs_mask, current_dynamics, fix_shared_components=False):
        batch_size, num_candidate = candidate_representations.size(0), candidate_representations.size(1)

        aggregated_obs_representation = self.masked_mean(h_obs, obs_mask)

        if self.recurrent:
            aggregated_obs_representation = self.dynamics_aggregation(
                torch.cat([aggregated_obs_representation, current_dynamics], -1)
            )
            aggregated_obs_representation = torch.relu(aggregated_obs_representation)

        if fix_shared_components:
            aggregated_obs_representation = aggregated_obs_representation.detach()
            candidate_representations = candidate_representations.detach()

        new_h_expanded = torch.stack([aggregated_obs_representation] * num_candidate, 1)

        output = self.action_scorer_linear_1(
            torch.cat([candidate_representations, new_h_expanded], -1)
        )
        output = torch.relu(output)
        output = output * cand_mask.unsqueeze(-1)

        if fix_shared_components:
            cand_mask_squared = torch.bmm(cand_mask.unsqueeze(-1), cand_mask.unsqueeze(1))
            output, _ = self.action_scorer_extra_self_attention(output, cand_mask_squared, output, output)
            output = self.action_scorer_extra_linear(output)
            output = torch.relu(output)
            output = output * cand_mask.unsqueeze(-1)

        output = self.action_scorer_linear_2(output).squeeze(-1)
        output = output * cand_mask

        return output, cand_mask


print("✓ Policy 网络已定义")

## 9. Double DQN Agent 实现 🎮

In [ ]:
from collections import deque, namedtuple
import torch.optim as optim

Transition = namedtuple('Transition', [
    'obs', 'task', 'action_idx', 'reward', 'next_obs', 'next_task', 'done',
    'admissible', 'next_admissible', 'prev_dynamics'
])


class PrioritizedReplayBuffer:
    """Prioritized Experience Replay"""
    def __init__(self, capacity=500000, alpha=0.6):
        self.capacity = capacity
        self.alpha = alpha
        self.buffer = []
        self.priorities = np.zeros(capacity, dtype=np.float32)
        self.position = 0
        self.size = 0

    def push(self, *args):
        max_priority = self.priorities[:self.size].max() if self.size > 0 else 1.0

        if len(self.buffer) < self.capacity:
            self.buffer.append(Transition(*args))
        else:
            self.buffer[self.position] = Transition(*args)

        self.priorities[self.position] = max_priority
        self.position = (self.position + 1) % self.capacity
        self.size = min(self.size + 1, self.capacity)

    def sample(self, batch_size, beta=0.4):
        if self.size == 0:
            return [], [], []

        priorities = self.priorities[:self.size]
        probs = priorities ** self.alpha
        probs /= probs.sum()

        indices = np.random.choice(self.size, min(batch_size, self.size), p=probs, replace=False)
        samples = [self.buffer[idx] for idx in indices]

        total = self.size
        weights = (total * probs[indices]) ** (-beta)
        weights /= weights.max()

        return samples, indices, weights

    def update_priorities(self, indices, priorities):
        for idx, priority in zip(indices, priorities):
            self.priorities[idx] = priority

    def __len__(self):
        return self.size


class DoubleDQNAgent:
    """Double DQN Agent"""
    def __init__(self, online_net, target_net, tokenizer, word2id, device, config):
        self.online_net = online_net.to(device)
        self.target_net = target_net.to(device)
        self.target_net.load_state_dict(online_net.state_dict())
        self.target_net.eval()

        self.tokenizer = tokenizer
        self.word2id = word2id
        self.device = device
        self.config = config

        self.learning_rate = config['learning_rate']
        self.gamma = config['gamma']
        self.batch_size = config['batch_size']
        self.target_update_frequency = config['target_update_frequency']
        self.update_per_k_game_steps = config['update_per_k_game_steps']
        self.multi_step = config.get('multi_step', 3)

        self.epsilon_start = config['epsilon_start']
        self.epsilon_end = config['epsilon_end']
        self.epsilon_anneal_episodes = config['epsilon_anneal_episodes']
        self.epsilon = self.epsilon_start

        self.optimizer = optim.Adam(online_net.parameters(), lr=self.learning_rate)
        self.memory = PrioritizedReplayBuffer(capacity=config['replay_memory_capacity'])

        self.step_in_total = 0
        self.episode_no = 0
        self.mode = "train"

    def update_epsilon(self):
        if self.episode_no < self.epsilon_anneal_episodes:
            self.epsilon = self.epsilon_start - (self.epsilon_start - self.epsilon_end) * (self.episode_no / self.epsilon_anneal_episodes)
        else:
            self.epsilon = self.epsilon_end

    def words_to_ids(self, words):
        ids = self.tokenizer.encode(words, add_special_tokens=True)
        return torch.tensor([ids], dtype=torch.long).to(self.device)

    def encode(self, texts, use_model="online"):
        model = self.online_net if use_model == "online" else self.target_net

        input_ids_list = []
        for text in texts:
            ids = self.tokenizer.encode(text, add_special_tokens=True, max_length=512, truncation=True)
            input_ids_list.append(ids)

        max_len = max(len(ids) for ids in input_ids_list)
        input_ids_padded = [ids + [0] * (max_len - len(ids)) for ids in input_ids_list]
        input_ids = torch.tensor(input_ids_padded, dtype=torch.long).to(self.device)

        h, mask = model.encode_text(input_ids)
        return h, mask

    def choose_random_action(self, action_scores, action_candidate_list):
        batch_size = action_scores.size(0)
        indices = []
        for j in range(batch_size):
            indices.append(np.random.choice(len(action_candidate_list[j])))
        return np.array(indices)

    def choose_maxQ_action(self, action_scores, action_mask=None):
        action_scores = action_scores - torch.min(action_scores, -1, keepdim=True)[0] + 1e-2
        if action_mask is not None:
            action_scores = action_scores * action_mask
        action_indices = torch.argmax(action_scores, -1)
        return action_indices.cpu().numpy()

    def action_scoring(self, action_candidate_list, h_obs, obs_mask, h_td, td_mask, previous_dynamics, use_model="online"):
        model = self.online_net if use_model == "online" else self.target_net
        batch_size = len(action_candidate_list)

        aggregated_obs_representation = model.aggretate_information(h_obs, obs_mask, h_td, td_mask)

        if model.recurrent:
            averaged_representation = model.masked_mean(aggregated_obs_representation, obs_mask)
            current_dynamics = model.rnncell(averaged_representation, previous_dynamics) if previous_dynamics is not None else model.rnncell(averaged_representation)
        else:
            current_dynamics = None

        max_num_candidate = max(len(candidates) for candidates in action_candidate_list)
        candidate_representations_list = []
        candidate_masks_list = []

        for candidates in action_candidate_list:
            if len(candidates) == 0:
                candidate_reps = torch.zeros(max_num_candidate, model.block_hidden_dim).to(self.device)
                candidate_mask = torch.zeros(max_num_candidate).to(self.device)
            else:
                h_cand, cand_mask = self.encode(candidates, use_model=use_model)
                candidate_reps = model.masked_mean(h_cand, cand_mask)

                if len(candidates) < max_num_candidate:
                    padding = torch.zeros(max_num_candidate - len(candidates), model.block_hidden_dim).to(self.device)
                    candidate_reps = torch.cat([candidate_reps, padding], dim=0)

                    mask_padding = torch.zeros(max_num_candidate - len(candidates)).to(self.device)
                    candidate_mask = torch.cat([cand_mask.sum(dim=1) > 0, mask_padding.bool()], dim=0).float()
                else:
                    candidate_mask = (cand_mask.sum(dim=1) > 0).float()

            candidate_representations_list.append(candidate_reps)
            candidate_masks_list.append(candidate_mask)

        candidate_representations = torch.stack(candidate_representations_list)
        candidate_masks = torch.stack(candidate_masks_list)

        action_scores, action_masks = model.score_actions(
            candidate_representations, candidate_masks,
            aggregated_obs_representation, obs_mask,
            current_dynamics, fix_shared_components=False
        )

        return action_scores, action_masks, current_dynamics

    def admissible_commands_act(self, obs_strings, task_strings, action_candidate_list, previous_dynamics, random=False):
        with torch.no_grad():
            if self.mode == "eval" or not random:
                h_obs, obs_mask = self.encode(obs_strings, use_model="online")
                h_td, td_mask = self.encode(task_strings, use_model="online")
                action_scores, action_masks, current_dynamics = self.action_scoring(
                    action_candidate_list, h_obs, obs_mask, h_td, td_mask, previous_dynamics, use_model="online"
                )
                action_indices = self.choose_maxQ_action(action_scores, action_masks)
                chosen_actions = [candidates[idx] for candidates, idx in zip(action_candidate_list, action_indices)]
                return chosen_actions, action_indices, current_dynamics

            batch_size = len(obs_strings)
            h_obs, obs_mask = self.encode(obs_strings, use_model="online")
            h_td, td_mask = self.encode(task_strings, use_model="online")
            action_scores, action_masks, current_dynamics = self.action_scoring(
                action_candidate_list, h_obs, obs_mask, h_td, td_mask, previous_dynamics, use_model="online"
            )

            action_indices_maxq = self.choose_maxQ_action(action_scores, action_masks)
            action_indices_random = self.choose_random_action(action_scores, action_candidate_list)

            rand_num = np.random.uniform(low=0.0, high=1.0, size=(batch_size,))
            less_than_epsilon = (rand_num < self.epsilon).astype("float32")
            greater_than_epsilon = 1.0 - less_than_epsilon

            chosen_indices = less_than_epsilon * action_indices_random + greater_than_epsilon * action_indices_maxq
            chosen_indices = chosen_indices.astype(int)
            chosen_actions = [candidates[idx] for candidates, idx in zip(action_candidate_list, chosen_indices)]

            return chosen_actions, chosen_indices, current_dynamics

    def update(self, beta=0.4):
        if len(self.memory) < self.batch_size:
            return None

        transitions, indices, weights = self.memory.sample(self.batch_size, beta=beta)
        batch = Transition(*zip(*transitions))
        weights = torch.FloatTensor(weights).to(self.device)

        batch_size = len(transitions)

        h_obs, obs_mask = self.encode(batch.obs, use_model="online")
        h_td, td_mask = self.encode(batch.task, use_model="online")
        action_scores, _, current_dynamics = self.action_scoring(
            batch.admissible, h_obs, obs_mask, h_td, td_mask, batch.prev_dynamics, use_model="online"
        )

        q_values = []
        for i in range(batch_size):
            q_values.append(action_scores[i, batch.action_idx[i]])
        q_values = torch.stack(q_values)

        next_q_values = []
        for i in range(batch_size):
            if batch.done[i]:
                next_q_values.append(torch.tensor(0.0).to(self.device))
            else:
                h_obs_next, obs_mask_next = self.encode([batch.next_obs[i]], use_model="online")
                h_td_next, td_mask_next = self.encode([batch.next_task[i]], use_model="online")

                aggregated_obs_next = self.online_net.aggretate_information(h_obs_next, obs_mask_next, h_td_next, td_mask_next)
                if self.online_net.recurrent:
                    averaged_next = self.online_net.masked_mean(aggregated_obs_next, obs_mask_next)
                    next_dynamics = self.online_net.rnncell(averaged_next, current_dynamics[i:i+1])
                else:
                    next_dynamics = None

                action_scores_online, _, _ = self.action_scoring(
                    [batch.next_admissible[i]], h_obs_next, obs_mask_next, h_td_next, td_mask_next,
                    next_dynamics, use_model="online"
                )
                best_action_idx = action_scores_online[0].argmax().item()

                action_scores_target, _, _ = self.action_scoring(
                    [batch.next_admissible[i]], h_obs_next, obs_mask_next, h_td_next, td_mask_next,
                    next_dynamics, use_model="target"
                )
                next_q_values.append(action_scores_target[0, best_action_idx])

        next_q_values = torch.stack(next_q_values)

        rewards = torch.tensor(batch.reward, dtype=torch.float32).to(self.device)
        target_q_values = rewards + (self.gamma ** self.multi_step) * next_q_values

        td_errors = torch.abs(q_values - target_q_values).detach().cpu().numpy()
        self.memory.update_priorities(indices, td_errors + 1e-6)

        loss = (weights * F.smooth_l1_loss(q_values, target_q_values, reduction='none')).mean()

        self.optimizer.zero_grad()
        loss.backward()
        torch.nn.utils.clip_grad_norm_(self.online_net.parameters(), max_norm=0.1)
        self.optimizer.step()

        return loss.item()

    def update_target_network(self):
        self.target_net.load_state_dict(self.online_net.state_dict())


print("✓ DoubleDQNAgent 已定义")

## 10. 训练循环实现 🔄

In [ ]:
from tqdm.notebook import tqdm
import logging

logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)


def process_ob(ob):
    """处理观察"""
    if ob.startswith('You arrive at loc '):
        ob = ob[ob.find('. ')+2:]
    return ob


def run_episode(env, agent, task_desc, max_steps=50, train=True):
    """运行一个 episode"""
    obs, info = env.reset()
    obs_text = '\n'.join(obs[0].split('\n\n')[1:])
    obs_text = process_ob(obs_text)

    total_reward = 0
    step = 0
    losses = []
    prev_dynamics = None

    while step < max_steps:
        if 'admissible_commands' not in info or len(info['admissible_commands']) == 0:
            break

        admissible = info['admissible_commands'][0]

        actions, action_indices, current_dynamics = agent.admissible_commands_act(
            [obs_text], [task_desc], [admissible], prev_dynamics, random=train
        )
        action = actions[0]
        action_idx = action_indices[0]

        next_obs, _, done, next_info = env.step([action])
        next_obs_text = process_ob(next_obs[0])
        reward = 1.0 if next_info['won'][0] else 0.0
        done = done[0]

        if train:
            next_admissible = next_info.get('admissible_commands', [[]])[0] if not done else []
            agent.memory.push(
                obs_text, task_desc, action_idx, reward, next_obs_text, task_desc,
                done, admissible, next_admissible, prev_dynamics
            )

            if agent.step_in_total % agent.update_per_k_game_steps == 0 and len(agent.memory) >= agent.batch_size:
                loss = agent.update(beta=0.4)
                if loss is not None:
                    losses.append(loss)

        total_reward += reward
        obs_text = next_obs_text
        info = next_info
        prev_dynamics = current_dynamics
        step += 1
        agent.step_in_total += 1

        if done:
            break

    agent.episode_no += 1
    agent.update_epsilon()

    if agent.episode_no % agent.target_update_frequency == 0:
        agent.update_target_network()

    avg_loss = np.mean(losses) if losses else 0.0
    return total_reward, step, avg_loss, done


print("✓ 训练函数已定义")

## 11. 训练配置 ⚙️

In [ ]:
from datetime import datetime

# ========== 训练参数 ==========
NUM_EPISODES = 100        # 快速测试，完整训练建议 1000-5000
EVAL_FREQ = 20            # 每 20 episodes 评估一次
SAVE_FREQ = 50            # 每 50 episodes 保存一次

# ========== 模型配置 ==========
MODEL_CONFIG = {
    'encoder_layers': 1,
    'encoder_conv_num': 5,
    'block_hidden_dim': 64,
    'n_heads': 1,
    'block_dropout': 0.1,
    'dropout': 0.1,
    'recurrent': True,
}

# ========== DQN 配置 ==========
TRAINING_CONFIG = {
    'learning_rate': 0.001,
    'gamma': 0.9,
    'batch_size': 64,
    'target_update_frequency': 500,
    'update_per_k_game_steps': 5,
    'multi_step': 3,
    'epsilon_start': 0.3,
    'epsilon_end': 0.1,
    'epsilon_anneal_episodes': 1000,
    'replay_memory_capacity': 100000,  # 减少以节省内存
}

# ========== 路径 ==========
timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
RUN_NAME = f"dqn_distilbert_{timestamp}"
SAVE_PATH = os.path.join(CHECKPOINTS_DIR, f"{RUN_NAME}.pt")
LOG_PATH = os.path.join(RESULTS_DIR, f"{RUN_NAME}_log.txt")

# ========== 显示配置 ==========
print("="*60)
print("📋 训练配置")
print("="*60)
print(f"运行名称: {RUN_NAME}")
print(f"Episodes: {NUM_EPISODES}")
print(f"Batch Size: {TRAINING_CONFIG['batch_size']}")
print(f"Learning Rate: {TRAINING_CONFIG['learning_rate']}")
print(f"Gamma: {TRAINING_CONFIG['gamma']}")
print(f"Epsilon: {TRAINING_CONFIG['epsilon_start']} → {TRAINING_CONFIG['epsilon_end']}")
print(f"Hidden Dim: {MODEL_CONFIG['block_hidden_dim']}")
print(f"\n保存路径: {SAVE_PATH}")
print(f"日志路径: {LOG_PATH}")
print("="*60)

# 时间估算
time_min = NUM_EPISODES * 1
time_max = NUM_EPISODES * 3
print(f"\n⏱️ 预估时间: {time_min}-{time_max} 分钟")
print(f"💰 成本: $0")

## 12. 创建网络和 Agent 🤖

In [ ]:
print("创建 DQN Agent...\n")

# 创建网络
word_vocab_size = len(word2id)
online_net = Policy(bert_model, word_vocab_size, MODEL_CONFIG)
target_net = Policy(bert_model, word_vocab_size, MODEL_CONFIG)

print(f"✓ Policy 网络已创建")
print(f"  参数量: {sum(p.numel() for p in online_net.parameters()) / 1e6:.2f}M")

# 创建 Agent
agent = DoubleDQNAgent(
    online_net=online_net,
    target_net=target_net,
    tokenizer=tokenizer,
    word2id=word2id,
    device=device,
    config=TRAINING_CONFIG
)

print(f"\n✓ DoubleDQNAgent 已创建")
print(f"  设备: {device}")
print(f"  Replay Buffer: {TRAINING_CONFIG['replay_memory_capacity']}")

if torch.cuda.is_available():
    print(f"\nGPU 显存使用: {torch.cuda.memory_allocated() / 1024**3:.2f} GB")

## 13. 开始训练 🚀

In [ ]:
import alfworld.agents.environment
import yaml

print("="*60)
print("🚀 开始 DQN 训练")
print("="*60)

# 加载配置
with open(config_path) as f:
    env_config = yaml.safe_load(f)

# 创建环境
print("\n[1/3] 创建训练环境...")
train_env = alfworld.agents.environment.get_environment(env_config["env"]["type"])(env_config, train_eval='train')
train_env = train_env.init_env(batch_size=1)
print("✓ 训练环境已创建")

print("\n[2/3] 创建评估环境...")
eval_env = alfworld.agents.environment.get_environment(env_config["env"]["type"])(env_config, train_eval='eval_out_of_distribution')
eval_env = eval_env.init_env(batch_size=1)
print("✓ 评估环境已创建")

print("\n[3/3] 初始化训练循环...")
task_desc = "Your task is to: put some object on some receptacle."

train_rewards = []
train_steps = []
train_losses = []
eval_rewards = []
eval_success_rates = []

print("✓ 准备完成\n")
print("="*60)
print("开始训练...")
print("="*60)

In [ ]:
# 训练循环
try:
    for episode in tqdm(range(NUM_EPISODES), desc="训练进度"):
        # 训练
        reward, steps, loss, success = run_episode(train_env, agent, task_desc, train=True)
        train_rewards.append(reward)
        train_steps.append(steps)
        train_losses.append(loss)

        # 评估
        if (episode + 1) % EVAL_FREQ == 0:
            eval_reward_list = []
            eval_success_list = []

            agent.mode = "eval"
            for _ in range(5):
                eval_reward, _, _, eval_success = run_episode(eval_env, agent, task_desc, train=False)
                eval_reward_list.append(eval_reward)
                eval_success_list.append(eval_success)
            agent.mode = "train"

            avg_eval_reward = np.mean(eval_reward_list)
            avg_eval_success = np.mean(eval_success_list)
            eval_rewards.append(avg_eval_reward)
            eval_success_rates.append(avg_eval_success)

            print(f"\n{'='*60}")
            print(f"Episode {episode+1}/{NUM_EPISODES}")
            print(f"  训练奖励: {np.mean(train_rewards[-EVAL_FREQ:]):.3f}")
            print(f"  评估奖励: {avg_eval_reward:.3f}")
            print(f"  评估成功率: {avg_eval_success:.1%}")
            print(f"  Epsilon: {agent.epsilon:.3f}")
            print(f"  Loss: {loss:.4f}")
            print(f"  缓冲区: {len(agent.memory)}")
            if torch.cuda.is_available():
                print(f"  GPU显存: {torch.cuda.memory_allocated()/1024**3:.2f}GB")
            print(f"{'='*60}")

        # 保存检查点
        if (episode + 1) % SAVE_FREQ == 0:
            checkpoint_path = os.path.join(CHECKPOINTS_DIR, f"{RUN_NAME}_ep{episode+1}.pt")
            torch.save({
                'episode': episode + 1,
                'online_net': online_net.state_dict(),
                'target_net': target_net.state_dict(),
                'optimizer': agent.optimizer.state_dict(),
                'train_rewards': train_rewards,
                'eval_rewards': eval_rewards,
                'config': {**MODEL_CONFIG, **TRAINING_CONFIG}
            }, checkpoint_path)
            print(f"\n💾 检查点已保存: {checkpoint_path}")

    print("\n" + "="*60)
    print("✅ 训练完成！")
    print("="*60)

except KeyboardInterrupt:
    print("\n⚠️ 训练被中断")
except Exception as e:
    print(f"\n❌ 训练错误: {e}")
    import traceback
    traceback.print_exc()
finally:
    # 保存最终模型
    torch.save({
        'episode': len(train_rewards),
        'online_net': online_net.state_dict(),
        'target_net': target_net.state_dict(),
        'optimizer': agent.optimizer.state_dict(),
        'train_rewards': train_rewards,
        'eval_rewards': eval_rewards,
        'config': {**MODEL_CONFIG, **TRAINING_CONFIG}
    }, SAVE_PATH)
    print(f"\n💾 最终模型已保存: {SAVE_PATH}")
    
    train_env.close()
    eval_env.close()

## 14. 结果分析 📊

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

# 统计信息
print("\n📊 DQN 训练结果")
print("="*60)
print(f"总 Episodes: {len(train_rewards)}")
print(f"平均训练奖励: {np.mean(train_rewards):.3f}")
print(f"最终训练奖励: {np.mean(train_rewards[-20:]):.3f}")
if eval_rewards:
    print(f"平均评估奖励: {np.mean(eval_rewards):.3f}")
    print(f"最终评估奖励: {eval_rewards[-1]:.3f}")
if eval_success_rates:
    print(f"平均成功率: {np.mean(eval_success_rates):.1%}")
    print(f"最终成功率: {eval_success_rates[-1]:.1%}")
print(f"平均 Loss: {np.mean([l for l in train_losses if l > 0]):.4f}")
print("="*60)

In [ ]:
# 可视化
sns.set_style("whitegrid")
fig, axes = plt.subplots(2, 2, figsize=(14, 10))

# 1. 训练奖励
axes[0, 0].plot(train_rewards, alpha=0.3, label='Raw', color='#2E86AB')
if len(train_rewards) >= 20:
    smoothed = pd.Series(train_rewards).rolling(20).mean()
    axes[0, 0].plot(smoothed, lw=2, label='Smoothed (20)', color='#2E86AB')
axes[0, 0].set_xlabel('Episode', fontsize=11, weight='bold')
axes[0, 0].set_ylabel('Reward', fontsize=11, weight='bold')
axes[0, 0].set_title('Training Rewards', fontsize=12, weight='bold')
axes[0, 0].legend()
axes[0, 0].grid(alpha=0.3)

# 2. 评估奖励
if eval_rewards:
    eval_x = list(range(EVAL_FREQ, len(train_rewards)+1, EVAL_FREQ))[:len(eval_rewards)]
    axes[0, 1].plot(eval_x, eval_rewards, 'o-', lw=2, ms=6, color='#06A77D')
    axes[0, 1].set_xlabel('Episode', fontsize=11, weight='bold')
    axes[0, 1].set_ylabel('Avg Reward', fontsize=11, weight='bold')
    axes[0, 1].set_title('Evaluation Rewards', fontsize=12, weight='bold')
    axes[0, 1].grid(alpha=0.3)

# 3. Loss
valid_losses = [(i, l) for i, l in enumerate(train_losses) if l > 0]
if valid_losses:
    loss_x, loss_y = zip(*valid_losses)
    axes[1, 0].plot(loss_x, loss_y, alpha=0.3, color='#A23B72')
    if len(loss_y) >= 50:
        smoothed_loss = pd.Series(loss_y).rolling(50).mean()
        axes[1, 0].plot(loss_x, smoothed_loss, lw=2, color='#A23B72')
axes[1, 0].set_xlabel('Episode', fontsize=11, weight='bold')
axes[1, 0].set_ylabel('Loss', fontsize=11, weight='bold')
axes[1, 0].set_title('DQN Loss', fontsize=12, weight='bold')
axes[1, 0].grid(alpha=0.3)

# 4. 成功率
if eval_success_rates:
    eval_x = list(range(EVAL_FREQ, len(train_rewards)+1, EVAL_FREQ))[:len(eval_success_rates)]
    axes[1, 1].plot(eval_x, [s*100 for s in eval_success_rates], 'o-', lw=2, ms=6, color='#F18F01')
    axes[1, 1].set_xlabel('Episode', fontsize=11, weight='bold')
    axes[1, 1].set_ylabel('Success Rate (%)', fontsize=11, weight='bold')
    axes[1, 1].set_title('Evaluation Success Rate', fontsize=12, weight='bold')
    axes[1, 1].set_ylim([0, 105])
    axes[1, 1].grid(alpha=0.3)

plt.tight_layout()
plt.savefig(os.path.join(RESULTS_DIR, f'{RUN_NAME}_curves.png'), dpi=150, bbox_inches='tight')
plt.show()

print(f"\n✓ 图表已保存: {os.path.join(RESULTS_DIR, f'{RUN_NAME}_curves.png')}")

## 15. 测试训练好的 Agent 🎮

In [ ]:
print("测试训练好的 Agent...\n")

# 创建测试环境
test_env = alfworld.agents.environment.get_environment(env_config["env"]["type"])(env_config, train_eval='eval_out_of_distribution')
test_env = test_env.init_env(batch_size=1)

# 设置为评估模式
agent.mode = "eval"

# 运行测试
print("运行 10 个测试 episodes...\n")
test_results = []

for i in range(10):
    reward, steps, _, success = run_episode(test_env, agent, task_desc, train=False)
    test_results.append({
        'episode': i+1,
        'reward': reward,
        'steps': steps,
        'success': success
    })
    print(f"Episode {i+1}: Reward={reward:.1f}, Steps={steps}, Success={success}")

test_env.close()

# 统计
df_test = pd.DataFrame(test_results)
print("\n" + "="*60)
print("测试结果")
print("="*60)
print(f"平均奖励: {df_test['reward'].mean():.3f}")
print(f"成功率: {df_test['success'].mean():.1%}")
print(f"平均步数: {df_test['steps'].mean():.1f}")
print("="*60)

## 16. 保存结果到 Drive 💾

In [ ]:
import json

# 保存训练历史
history = {
    'train_rewards': train_rewards,
    'train_steps': train_steps,
    'train_losses': train_losses,
    'eval_rewards': eval_rewards,
    'eval_success_rates': eval_success_rates,
    'config': {
        'model': MODEL_CONFIG,
        'training': TRAINING_CONFIG,
        'num_episodes': NUM_EPISODES
    }
}

history_path = os.path.join(RESULTS_DIR, f'{RUN_NAME}_history.json')
with open(history_path, 'w') as f:
    json.dump(history, f, indent=2)

print(f"✓ 训练历史已保存: {history_path}")

# 创建总结
summary = f"""
# DQN Training Summary

## Configuration
- Model: DistilBERT + Double DQN
- Episodes: {NUM_EPISODES}
- Hidden Dim: {MODEL_CONFIG['block_hidden_dim']}
- Learning Rate: {TRAINING_CONFIG['learning_rate']}
- Batch Size: {TRAINING_CONFIG['batch_size']}
- Gamma: {TRAINING_CONFIG['gamma']}

## Results
- Final Training Reward: {np.mean(train_rewards[-20:]):.3f}
- Final Eval Reward: {eval_rewards[-1] if eval_rewards else 'N/A'}
- Final Success Rate: {eval_success_rates[-1]:.1%} if eval_success_rates else 'N/A'}
- Average Loss: {np.mean([l for l in train_losses if l > 0]):.4f}

## Files
- Model: {SAVE_PATH}
- History: {history_path}
- Curves: {os.path.join(RESULTS_DIR, f'{RUN_NAME}_curves.png')}

Generated: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}
"""

summary_path = os.path.join(RESULTS_DIR, f'{RUN_NAME}_summary.txt')
with open(summary_path, 'w') as f:
    f.write(summary)

print(f"✓ 总结已保存: {summary_path}")
print("\n" + summary)

## 17. 下载结果（可选） ⬇️

In [ ]:
if IN_COLAB:
    import shutil
    from google.colab import files
    
    # 压缩结果
    zip_path = f"/content/{RUN_NAME}"
    shutil.make_archive(zip_path, 'zip', RESULTS_DIR)
    
    print(f"压缩完成: {zip_path}.zip")
    print("\n下载中...")
    files.download(f"{zip_path}.zip")
    print("✓ 完成")
else:
    print(f"结果位置: {RESULTS_DIR}")

## 📝 使用说明

### 快速开始

1. **检查 GPU**：运行时 → 更改运行时类型 → GPU (T4)
2. **运行所有 cells**：首次约 20-30 分钟
3. **开始训练**：cell-13 开始训练循环

### 训练建议

**快速测试（1-2小时）**
```python
NUM_EPISODES = 100
EVAL_FREQ = 20
```

**中等训练（半天）**
```python
NUM_EPISODES = 500
EVAL_FREQ = 50
```

**完整训练（1-2天）**
```python
NUM_EPISODES = 2000
EVAL_FREQ = 100
```

### 恢复训练

```python
# 加载检查点
checkpoint = torch.load('/path/to/checkpoint.pt')
online_net.load_state_dict(checkpoint['online_net'])
target_net.load_state_dict(checkpoint['target_net'])
agent.optimizer.load_state_dict(checkpoint['optimizer'])
```

### 性能优化

如果显存不足：
- 减小 `batch_size`: 64 → 32
- 减小 `replay_memory_capacity`: 100000 → 50000
- 减小 `block_hidden_dim`: 64 → 32

### 问题排查

**环境初始化失败**
- 检查 `ALFWORLD_DATA` 环境变量
- 重新运行 cell-4 下载数据

**显存溢出**
- 减小 batch_size
- 减小 replay buffer 大小

**训练不收敛**
- 增加训练 episodes
- 调整学习率
- 检查 epsilon 衰减